* **Bayes’s Rule**:  the posterior odds are the prior odds times the likelihood ratio
* **Bayes factor**: ratio of the posterior to prior odds, which is equal to the likelihood ratio. If Bayes factor is greater than 1, then the data has made the event *more* likely.


**Exercise 1**: Let’s use Bayes’s Rule to solve the Elvis problem from <<_Distributions>>:

    Elvis Presley had a twin brother who died at birth. What is the probability that Elvis was an identical twin?

In 1935, about 2/3 of twins were fraternal and 1/3 were identical. The question contains two pieces of information we can use to update this prior.

First, Elvis’s twin was also male, which is more likely if they were identical twins, with a likelihood ratio of 2.

Also, Elvis’s twin died at birth, which is more likely if they were identical twins, with a likelihood ratio of 1.25.

If you are curious about where those numbers come from, I wrote a __[blog post](https://www.allendowney.com/blog/2020/01/28/the-elvis-problem-revisited)__ about it.

In [5]:
import numpy as np
from empiricaldist import Pmf
from scipy.stats import binom

# 1/3 of twins in 1935 were identical, so prior odds are 2:1 against.
prior_odds = 1/2

# Update using the two likelihood ratios
post_odds = prior_odds * 2 * 1.25

post_odds, post_odds / (post_odds + 1)

# The posterior odds are 1.25 in favor, or probability about 56%

(1.25, 0.5555555555555556)

**Exercise 2**: The following is an __[interview question that appeared on glassdoor.com](https://www.glassdoor.com/Interview/You-re-about-to-get-on-a-plane-to-Seattle-You-want-to-know-if-you-should-bring-an-umbrella-You-call-3-random-friends-of-y-QTN_519262.htm)__, attributed to Facebook:

    You’re about to get on a plane to Seattle. You want to know if you should bring an umbrella. You call 3 random friends of yours who live there and ask each independently if it’s raining. Each of your friends has a 2/3 chance of telling you the truth and a 1/3 chance of messing with you by lying. All 3 friends tell you that “Yes” it is raining. What is the probability that it’s actually raining in Seattle?

Use Bayes’s Rule to solve this problem. As a prior you can assume that it rains in Seattle about 10% of the time.

This question causes some confusion about the differences between Bayesian and frequentist interpretations of probability; if you are curious about this point, __[I wrote a blog article about it](http://allendowney.blogspot.com/2016/09/bayess-theorem-is-not-optional.html)__.

In [10]:
# if it rains in Seattle 10% of the time, prior odds in favor are 1 : 9
prior_odds = 1 / 9

# calculate the likelihoods of "raining" and "not raining" to get the likelihood ration
likelihood_rain = (2/3) ** 3

likelihood_not_rain = (1/3) ** 3

likelihood_ratio = likelihood_rain / likelihood_not_rain

post_odds = prior_odds * likelihood_ratio

post_odds, post_odds / (1 + post_odds)

# So the posterior odds are 8 : 9 in favor, or a little bit under even

(0.8888888888888888, 0.47058823529411764)

**Exercise 3**: __[According to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/health_effects/effects_cig_smoking)__, people who smoke are about 25 times more likely to develop lung cancer than nonsmokers.

__[Also according to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm)__, about 14% of adults in the U.S. are smokers. If you learn that someone has lung cancer, what is the probability they are a smoker?

In [14]:
prior_odds = 0.14 / (1 - 0.14)

likelihood_ratio = 25

posterior_odds = prior_odds * likelihood_ratio

posterior_odds, posterior_odds / (1 + posterior_odds)

(4.069767441860465, 0.8027522935779816)

**Exercise 4**: In *Dungeons & Dragons*, the amount of damage a goblin can withstand is the sum of two six-sided dice. The amount of damage you inflict with a short sword is determined by rolling one six-sided die. A goblin is defeated if the total damage you inflict is greater than or equal to the amount it can withstand.

Suppose you are fighting a goblin and you have already inflicted 3 points of damage. What is your probability of defeating the goblin with your next successful attack?

Hint: You can use `Pmf.add_dist` to add a constant amount, like 3, to a `Pmf` and `Pmf.sub_dist` to compute the distribution of remaining points.

In [56]:
# Define a fair six-sided die, as a PMF
die = Pmf(1/6, np.arange(1, 7))

# Use the add_dist function defined in the chapter
def add_dist(pmf1, pmf2):
    """Compute the distribution of a sum."""
    res = Pmf()
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            q = q1 + q2
            p = p1 * p2
            res[q] = res(q) + p
    return res

goblin_hp = add_dist(die, die)

# Left over health is original HP - 3
goblin_health = goblin_hp.sub_dist(3)

goblin_health[goblin_health.qs <= 0] = 0

# THere's no chance remaining health will be 0 or less
goblin_health.normalize()

health_after_second_hit = goblin_health.sub_dist(die)

health_after_second_hit.prob_le(0)
# the goblin's HP has to be greater than 3, or else it would've died with the first hit

##impossible = (3 >= goblin_hp.qs)

# goblin_hp[impossible] = 0

# goblin_hp.normalize()

# Now we can subtract the one-roll die PMF from goblin_hp, and look at the probability that the resulting PMF is less than or equal to 0

# goblin_hp = goblin_hp.sub_dist(3)

# goblin_hp.qs


0.4545454545454545